In [ ]:
import geopandas as gpd
import pandas as pd
import pathlib

import matplotlib.pyplot as plt

from cartopy import crs as ccrs

import os
import sys
import subprocess
import datetime
import platform
import datetime
from folium.plugins import StripePattern

In [ ]:
import glob
path=r'census_shp'

file=glob.glob(os.path.join(path, 'cb_2018_**_tract_500k.shp'))
# print(file)
print(len(file))

In [ ]:
basemap= gpd.GeoDataFrame()
for f in file:
    basemap = pd.concat([basemap, gpd.read_file(f)], axis=0)

In [ ]:
basemap = basemap[['GEOID','AFFGEOID','STATEFP','geometry']]
basemap.head()

In [ ]:
basemap.crs

In [ ]:
basemap=basemap.to_crs("ESRI:102003")

In [ ]:
basemap["geometry"] = basemap["geometry"].simplify(5000)

In [ ]:
basemap=basemap.to_crs("epsg:4326")

In [ ]:
basemap.crs

In [ ]:
us_jason = basemap.to_json()

In [ ]:
us_jason

# Folium

In [ ]:
exp = pd.read_csv('census_data/exposure_census.csv', dtype = {'State':str,"GEOID":str})
exp = exp.rename(columns = {'min-max':'Exposure Score'})
exp = exp[['GEOID', 'Exposure Score']]
exp.head()

In [ ]:
sens = pd.read_csv("census_data/sensitivity_census.csv", dtype={"Geocode": str})
sens = sens.rename(columns = {'min_max_sens':'Sensitivity Score', 'Geocode':'GEOID'})
sens = sens[['GEOID', 'Sensitivity Score']]
sens.head()

In [ ]:
ac = pd.read_csv('census_data/adaptive_capacity_census.csv', dtype={"GEO_ID":str})
ac = ac.rename(columns = {'AC_score':'Adaptive Capacity Score', 'GEO_ID':'GEOID'})
ac = ac[['GEOID', 'Adaptive Capacity Score']]
ac.head()

In [ ]:
vul = pd.read_csv('Vulnerability Score.csv', dtype={"GEOID": str})
vul = vul.rename(columns = {'vul_score_mulpi':'Multiplicative Vulnerability', 
                            'vul_score_add':'Additive Vulnerability'})
vul = vul[['GEOID', 'Multiplicative Vulnerability', 'Additive Vulnerability']]
vul.head()

In [ ]:
folium_data = pd.merge(exp, sens, on='GEOID',how='outer')
folium_data = pd.merge(folium_data, ac,on='GEOID',how='outer')
folium_data = pd.merge(folium_data, vul,on='GEOID',how='outer')
folium_data = pd.merge(folium_data, basemap,on='GEOID',how='right')
folium_data.head()

In [ ]:
folium_map = gpd.GeoDataFrame(folium_data,crs=4326)

In [ ]:
folium_map.info()

In [ ]:
import jenkspy
exp_data = folium_data['Exposure Score'].fillna(0, inplace=False)
sens_data = folium_data['Sensitivity Score'].fillna(0, inplace=False)
ac_data = folium_data['Adaptive Capacity Score'].fillna(0, inplace=False)
add_vul_data = folium_data['Additive Vulnerability'].fillna(0, inplace=False)
multi_vul_data = folium_data['Multiplicative Vulnerability'].fillna(0, inplace=False)

exp_bins = jenkspy.jenks_breaks(exp_data, nb_class=5)
sens_bins = jenkspy.jenks_breaks(sens_data, nb_class=5)
ac_bins = jenkspy.jenks_breaks(ac_data, nb_class=5)
add_vul_bins = jenkspy.jenks_breaks(add_vul_data, nb_class=5)
multi_vul_bins = jenkspy.jenks_breaks(multi_vul_data, nb_class=5)

In [ ]:
print(exp_bins, '\n', sens_bins, '\n', ac_bins, '\n', add_vul_bins, '\n', multi_vul_bins)

In [ ]:
def add_categorical_legend(folium_map, title, colors, labels):
    if len(colors) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    color_by_label = dict(zip(labels, colors))
    
    legend_categories = ""     
    for label, color in color_by_label.items():
        legend_categories += f"<li><span style='background:{color}'></span>{label}</li>"
        
    legend_html = f"""
    <div id='maplegend' class='maplegend'>
      <div class='legend-title'>{title}</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
        {legend_categories}
        </ul>
      </div>
    </div>
    """
    script = f"""
        <script type="text/javascript">
        var oneTimeExecution = (function() {{
                    var executed = false;
                    return function() {{
                        if (!executed) {{
                             var checkExist = setInterval(function() {{
                                       if ((document.getElementsByClassName('leaflet-top leaflet-right').length) || (!executed)) {{
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.display = "flex"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].style.flexDirection = "column"
                                          document.getElementsByClassName('leaflet-top leaflet-right')[0].innerHTML += `{legend_html}`;
                                          clearInterval(checkExist);
                                          executed = true;
                                       }}
                                    }}, 100);
                        }}
                    }};
                }})();
        oneTimeExecution()
        </script>
      """
   

    css = """

    <style type='text/css'>
      .maplegend {
        z-index:9999;
        float:right;
        background-color: rgba(255, 255, 255, 1);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:10px;
        positon: relative;
      }
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 12px;
        width: 32px;
        margin-right: 10px;
        margin-left: 10px;
        border: 0px solid #ccc;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    """

    folium_map.get_root().header.add_child(folium.Element(script + css))

    return folium_map

In [ ]:
import folium

m = folium.Map(location=[37, -102], zoom_start=4)

### Exposure map
c = folium.Choropleth(
    geo_data=us_jason,
    name='Exposure',
    data=folium_map,
    columns=['GEOID','Exposure Score'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = "White",
    legend_name='Exposure',
    bins=exp_bins,
)
for key in c._children:
    if key.startswith('color_map'):
        del(c._children[key])
c.add_to(m)

### Sensitivity map
d = folium.Choropleth(
    geo_data=us_jason,
    name='Sensitivity',
    data=folium_map,
    columns=['GEOID','Sensitivity Score'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = "White",
    legend_name='Sensitivity',
    bins=sens_bins,
)
for key in d._children:
    if key.startswith('color_map'):
        del(d._children[key])
d.add_to(m)

### Adaptive Capacity map
e = folium.Choropleth(
    geo_data=us_jason,
    name='Adaptive Capacity',
    data=folium_map,
    columns=['GEOID','Adaptive Capacity Score'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = "White",
    legend_name='Adaptive Capacity',
    bins=ac_bins,
)
for key in e._children:
    if key.startswith('color_map'):
        del(e._children[key])
e.add_to(m)

### Additive Vulnerability map
f = folium.Choropleth(
    geo_data=us_jason,
    name='Additive Vulnerability',
    data=folium_map,
    columns=['GEOID','Additive Vulnerability'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = "White",
    legend_name='Additive Vulnerability',
    bins=add_vul_bins,
)
for key in f._children:
    if key.startswith('color_map'):
        del(f._children[key])
f.add_to(m)

### Multiplicative Vulnerability map
g = folium.Choropleth(
    geo_data=us_jason,
    name='Multiplicative Vulnerability',
    data=folium_map,
    columns=['GEOID','Multiplicative Vulnerability'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = "White",
    legend_name='Multiplicative Vulnerability',
    bins=multi_vul_bins,
)
for key in g._children:
    if key.startswith('color_map'):
        del(g._children[key])
g.add_to(m)

# Add hover functionality.
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    data = folium_map,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Exposure Score', 'Sensitivity Score', 'Adaptive Capacity Score',
                'Additive Vulnerability', 'Multiplicative Vulnerability'],
        aliases=['Exposure Score', 'Sensitivity Score', 'Adaptive Capacity Score',
                 'Additive Vulnerability', 'Multiplicative Vulnerability'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(NIL)
m.keep_in_front(NIL)

# Add the customized legend
m = add_categorical_legend(m, 'Exposure',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['0.0 - 0.0437', '0.0437 - 0.0769', '0.0769 - 0.1130', '0.1130 - 0.1666', '0.1666 - 1.0'])
m = add_categorical_legend(m, 'Sensitivity',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['0.0 - 0.2272', '0.2272 - 0.4622', '0.4622 - 0.5427', '0.5427 - 0.6505', '0.6505 - 1.0'])
m = add_categorical_legend(m, 'Adaptive Capacity',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['0.0 - 0.0990', '0.0990 - 0.2500', '0.2500 - 0.3431', '0.3431 - 0.4239', '0.4239 - 1.0'])
m = add_categorical_legend(m, 'Additive Vulnerability',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['-0.6505 - 0.1707', '0.1707 - 0.3860', '0.3860 - 0.5557', '0.5557 - 0.7037', '0.7037 - 1.4384'])
m = add_categorical_legend(m, 'Multiplicative Vulnerability',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['-0.0935 - 0.0167', '0.0167 - 0.0368', '0.0368 - 0.0604', '0.0604 - 0.1000', '0.1000 - 0.4384'])



# Add cross-hatching (crossing lines) to display the Null values.
# nans = exposure_map[exposure_map["Exposure Score"].isnull()]['GEOID'].values
# gdf_nans = exposure_map[exposure_map['GEOID'].isin(nans)]
# sp = StripePattern(angle=45, color='grey', space_color='white')
# sp.add_to(m)
# folium.features.GeoJson(name="NaN values",
#                         data=gdf_nans, 
#                         style_function=lambda x :{'fillPattern': sp},show=False).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m.save('Folium_all.html')

In [ ]:

m = folium.Map(location=[37, -102], zoom_start=4)

### Exposure map
c = folium.Choropleth(
    geo_data=us_jason,
    name='Exposure',
    data=folium_map,
    columns=['GEOID','Exposure Score'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = "White",
    legend_name='Exposure',
    bins=exp_bins,
)
for key in c._children:
    if key.startswith('color_map'):
        del(c._children[key])
c.add_to(m)

# Add hover functionality.
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    data = folium_map,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Exposure Score', 'Sensitivity Score', 'Adaptive Capacity Score',
                'Additive Vulnerability', 'Multiplicative Vulnerability'],
        aliases=['Exposure Score', 'Sensitivity Score', 'Adaptive Capacity Score',
                 'Additive Vulnerability', 'Multiplicative Vulnerability'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(NIL)
m.keep_in_front(NIL)

# Add the customized legend
m = add_categorical_legend(m, 'Exposure',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['0.0 - 0.0437', '0.0437 - 0.0769', '0.0769 - 0.1130', '0.1130 - 0.1666', '0.1666 - 1.0'])

# Add cross-hatching (crossing lines) to display the Null values.
# nans = exposure_map[exposure_map["Exposure Score"].isnull()]['GEOID'].values
# gdf_nans = exposure_map[exposure_map['GEOID'].isin(nans)]
# sp = StripePattern(angle=45, color='grey', space_color='white')
# sp.add_to(m)
# folium.features.GeoJson(name="NaN values",
#                         data=gdf_nans, 
#                         style_function=lambda x :{'fillPattern': sp},show=False).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m.save('Folium_exp.html')

In [ ]:
m = folium.Map(location=[37, -102], zoom_start=4)

### Sensitivity map
d = folium.Choropleth(
    geo_data=us_jason,
    name='Sensitivity',
    data=folium_map,
    columns=['GEOID','Sensitivity Score'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = "White",
    legend_name='Sensitivity',
    bins=sens_bins,
)
for key in d._children:
    if key.startswith('color_map'):
        del(d._children[key])
d.add_to(m)

### Adaptive Capacity map
e = folium.Choropleth(
    geo_data=us_jason,
    name='Adaptive Capacity',
    data=folium_map,
    columns=['GEOID','Adaptive Capacity Score'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = "White",
    legend_name='Adaptive Capacity',
    bins=ac_bins,
)
for key in e._children:
    if key.startswith('color_map'):
        del(e._children[key])
e.add_to(m)

### Additive Vulnerability map
f = folium.Choropleth(
    geo_data=us_jason,
    name='Additive Vulnerability',
    data=folium_map,
    columns=['GEOID','Additive Vulnerability'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = "White",
    legend_name='Additive Vulnerability',
    bins=add_vul_bins,
)
for key in f._children:
    if key.startswith('color_map'):
        del(f._children[key])
f.add_to(m)

### Multiplicative Vulnerability map
g = folium.Choropleth(
    geo_data=us_jason,
    name='Multiplicative Vulnerability',
    data=folium_map,
    columns=['GEOID','Multiplicative Vulnerability'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_color="white",
    line_opacity=0,
    nan_fill_color = "White",
    legend_name='Multiplicative Vulnerability',
    bins=multi_vul_bins,
)
for key in g._children:
    if key.startswith('color_map'):
        del(g._children[key])
g.add_to(m)

# Add hover functionality.
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    data = folium_map,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Exposure Score', 'Sensitivity Score', 'Adaptive Capacity Score',
                'Additive Vulnerability', 'Multiplicative Vulnerability'],
        aliases=['Exposure Score', 'Sensitivity Score', 'Adaptive Capacity Score',
                 'Additive Vulnerability', 'Multiplicative Vulnerability'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(NIL)
m.keep_in_front(NIL)

# Add the customized legend
m = add_categorical_legend(m, 'Exposure',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['0.0 - 0.0437', '0.0437 - 0.0769', '0.0769 - 0.1130', '0.1130 - 0.1666', '0.1666 - 1.0'])
m = add_categorical_legend(m, 'Sensitivity',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['0.0 - 0.2272', '0.2272 - 0.4622', '0.4622 - 0.5427', '0.5427 - 0.6505', '0.6505 - 1.0'])
m = add_categorical_legend(m, 'Adaptive Capacity',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['0.0 - 0.0990', '0.0990 - 0.2500', '0.2500 - 0.3431', '0.3431 - 0.4239', '0.4239 - 1.0'])
m = add_categorical_legend(m, 'Additive Vulnerability',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['-0.6505 - 0.1707', '0.1707 - 0.3860', '0.3860 - 0.5557', '0.5557 - 0.7037', '0.7037 - 1.4384'])
m = add_categorical_legend(m, 'Multiplicative Vulnerability',
                           colors = ['#fef0d9','#fdcc8a','#fc8d59','#e34a33', '#b30000'],
                           labels = ['-0.0935 - 0.0167', '0.0167 - 0.0368', '0.0368 - 0.0604', '0.0604 - 0.1000', '0.1000 - 0.4384'])



# Add cross-hatching (crossing lines) to display the Null values.
# nans = exposure_map[exposure_map["Exposure Score"].isnull()]['GEOID'].values
# gdf_nans = exposure_map[exposure_map['GEOID'].isin(nans)]
# sp = StripePattern(angle=45, color='grey', space_color='white')
# sp.add_to(m)
# folium.features.GeoJson(name="NaN values",
#                         data=gdf_nans, 
#                         style_function=lambda x :{'fillPattern': sp},show=False).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m.save('Folium_all.html')